# Imports And Libraries

In [1]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd
from boto import get_square_secret

# import os
import pickle
from google.oauth2.service_account import Credentials

In [2]:
get_square_secret()

'EAAAFO_dqOOnvRZCLDvOtKfBGznlxPy5zb4anviDPIJBjXjXGkmo5d6w1jxoxtCn'

# Generic Functions

In [3]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

# Get Existing Subs

In [4]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)


HttpError: <HttpError 403 when requesting https://sheets.googleapis.com/v4/spreadsheets/1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8/values/Square%20Subs%21A%3AG?alt=json returned "The caller does not have permission". Details: "The caller does not have permission">

In [5]:
exist

NameError: name 'exist' is not defined

# Get Data of most Recent Sub

In [ ]:
dates = exist[exist['Date'].str.contains('T')]

most_recent_transaction_date = pd.to_datetime(dates['Date'], format='mixed').max().isoformat()

most_recent_transaction_date

'2023-10-19T10:06:28+00:00'

In [ ]:
most_recent_transaction_date='2024-03-18T15:12:20.000Z'

In [ ]:
most_recent_transaction_date = '2024-01-26T20:35:58.000Z'

# Get New Subs

In [ ]:
client = Client(
    access_token= 'EAAAFO_dqOOnvRZCLDvOtKfBGznlxPy5zb4anviDPIJBjXjXGkmo5d6w1jxoxtCn',#get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [ ]:
orders_api = client.orders
customers_api = client.customers

In [ ]:
most_recent_transaction_date
most_recent_transaction_date.split()

'2024-03-18T15:12:20.000Z'

In [102]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
    'query': {
        'filter': {
            
            'date_time_filter': {
                'created_at': {
                    'start_at': f'{most_recent_transaction_date}',
                    "end_at": "2024-03-21T15:12:20.000Z",
                }
            },
      # "state_filter": {
        # "states": [
        #   "COMPLETED"
        # ]
      # }
       },
    "sort": {
      "sort_field": "UPDATED_AT",
      "sort_order": "DESC"
    }
        
    },
   }

In [103]:
result = orders_api.search_orders(body)
s_orders = result.body['orders']


i=1
while result.cursor:
    print(i)
    i+=1
    result = orders_api.search_orders(body,
        cursor = result.cursor
    )
    s_orders.append(result.body['orders'])

    # customers.update(customers2)

In [104]:
s_orders = orders_api.search_orders(body).body['orders']

len(s_orders)

500

In [105]:
s_orders

[{'id': 'r43F70CmXvyTt9lNhlyidFdmaTKZY',
  'location_id': 'SBBJSGR24YHYB',
  'line_items': [{'uid': 'NpPqhuoa5QUyKyeA5tdUZD',
    'catalog_object_id': 'OKAX666RRB7XKNM6BJTRM6IM',
    'catalog_version': 1710781187965,
    'quantity': '1',
    'name': 'Match Sub',
    'variation_name': 'Full Game',
    'base_price_money': {'amount': 1000, 'currency': 'GBP'},
    'gross_sales_money': {'amount': 1000, 'currency': 'GBP'},
    'total_tax_money': {'amount': 0, 'currency': 'GBP'},
    'total_discount_money': {'amount': 0, 'currency': 'GBP'},
    'total_money': {'amount': 1000, 'currency': 'GBP'},
    'variation_total_price_money': {'amount': 1000, 'currency': 'GBP'},
    'item_type': 'ITEM',
    'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}],
  'created_at': '2024-03-18T21:28:12.758Z',
  'updated_at': '2024-03-26T12:01:09.000Z',
  'state': 'COMPLETED',
  'version': 7,
  'reference_id': '6c973b2c-e56e-11ee-a1b0-0229af516e6d',
  'total_tax_money': {'amount': 0, 'currency': 'GBP

In [106]:
s_orders

[{'id': 'r43F70CmXvyTt9lNhlyidFdmaTKZY',
  'location_id': 'SBBJSGR24YHYB',
  'line_items': [{'uid': 'NpPqhuoa5QUyKyeA5tdUZD',
    'catalog_object_id': 'OKAX666RRB7XKNM6BJTRM6IM',
    'catalog_version': 1710781187965,
    'quantity': '1',
    'name': 'Match Sub',
    'variation_name': 'Full Game',
    'base_price_money': {'amount': 1000, 'currency': 'GBP'},
    'gross_sales_money': {'amount': 1000, 'currency': 'GBP'},
    'total_tax_money': {'amount': 0, 'currency': 'GBP'},
    'total_discount_money': {'amount': 0, 'currency': 'GBP'},
    'total_money': {'amount': 1000, 'currency': 'GBP'},
    'variation_total_price_money': {'amount': 1000, 'currency': 'GBP'},
    'item_type': 'ITEM',
    'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}],
  'created_at': '2024-03-18T21:28:12.758Z',
  'updated_at': '2024-03-26T12:01:09.000Z',
  'state': 'COMPLETED',
  'version': 7,
  'reference_id': '6c973b2c-e56e-11ee-a1b0-0229af516e6d',
  'total_tax_money': {'amount': 0, 'currency': 'GBP

In [107]:
for x in s_orders:
    try:
        a = x['line_items']
    except:
        print(x)

{'id': 'BC1ztAAVTbM0Qx5JITk4yHqeV', 'location_id': 'SBBJSGR24YHYB', 'created_at': '2024-03-20T17:17:22Z', 'updated_at': '2024-03-20T17:17:22.000Z', 'state': 'COMPLETED', 'total_tax_money': {'amount': 0, 'currency': 'GBP'}, 'total_discount_money': {'amount': 0, 'currency': 'GBP'}, 'total_tip_money': {'amount': 0, 'currency': 'GBP'}, 'total_money': {'amount': 0, 'currency': 'GBP'}, 'closed_at': '2024-03-20T17:17:22.000Z', 'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}, 'returns': [{'source_order_id': 'HR9KK3QNcctEzHdsAXuiy30eV', 'return_line_items': [{'uid': '98F608F8-F44D-4C36-97CD-70865C3FE1BF', 'source_line_item_uid': 'C39B0612-6937-4C33-A2ED-A091E6CEDA2C', 'name': 'Guinness -Draft', 'quantity': '1', 'catalog_object_id': 'KBHCUDOJN57MJ37FMUIQ6YL2', 'catalog_version': 1710717483376, 'variation_name': 'Regular', 'item_type': 'ITEM', 'base_price_money': {'amount': 675, 'currency': 'GBP'}, 'variation_total_price_money': {'amount': 675, 'currency': 'GBP'}, 'gross_return_mon

#  Get All Invoices

In [108]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
    print(result.cursor)
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERuR01RZ254ZUYwUERoaWk3LXRleHpFSU1Q
2
CgUIgwcYAQonCIMPEiJpbnY6MC1DaER6UHY2THVSVU5tOFlkWm42bXNUX3pFSU1Q
3
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEItMEZ0WUVZNTNZX21pWHVvLVkxMExFSU1Q
4
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEF4bzhGZkVwZHdCMzZrcFVHQUpFelVFSU1Q
5
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEJDMWdUeFdvQ3BPWWtEWE5MY3JvRUdFSU1Q
6
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFwZUJVWnFsZUp3a19WSXBGYkxyZG9FSU1Q
7
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEJ5c2xtMVZ5a0h3dHQ1dGllbjZaY1pFSU1Q
8
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEMxcm9nWmhSaWNwQ09zRThOSFlONnVFSU1Q
9
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERobm9vUFVwSmxvMWUxOTlhSjZLanpFSU1Q
10
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEMyODJtcGE5UnZyYjhQTnhRWi1uaFNFSU1Q
11
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFNWnA1TUt0SFprZmJSb1IzT0ZYeFBFSU1Q
12
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERoLW50cEpxcEpMUVlOdk1vei1IWkFFSU1Q
13
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERzeWgzWjdtcXV5SXRPRWFGY2NsQ0JFSU1Q
14
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERwMkRyQ2RRUXVfcm9lQVU4TDNIa3BFSU1Q
15
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERPNWh4ZFRtU01SUERNNmJmcD

,id,version,location_id,payment_requests,primary_recipient,invoice_number,title,description,public_url,next_payment_amount_money,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,store_payment_method_enabled,order_id,sale_or_service_date
0,inv:0-ChD71_aQXwqW7kRXjMnTfVFWEIMP,0,SBBJSGR24YHYB,[{'uid': '845a83a2-27fa-4c92-9097-9df15b14d7fe...,"{'customer_id': '4BRYNYNMNPXHXXP6G0MSMJP46G', ...",000538,March Training,,https://squareup.com/pay-invoice/inv:0-ChD71_a...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2024-04-01T10:38:48Z,2024-04-01T10:38:50Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN
1,inv:0-ChBkDKro6ZKCUuE6T3gQ9Z74EIMP,0,SBBJSGR24YHYB,[{'uid': '1c66433d-ad92-4b73-9e61-7f9864d1cb01...,"{'customer_id': 'QD87PHCMYX60S6P5Y9JP4PZFS4', ...",000537,March Training,,https://squareup.com/pay-invoice/inv:0-ChBkDKr...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2024-04-01T10:38:16Z,2024-04-01T10:38:17Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN
2,inv:0-ChAp4MadplE0i69r5MF13DvSEIMP,2,SBBJSGR24YHYB,[{'uid': 'a56c50d5-d5ef-4f39-ab49-09aec4d3f19f...,"{'customer_id': '4X4156TXR0YQB7KWXSD47ST0SW', ...",000536,March Training,,https://squareup.com/pay-invoice/inv:0-ChAp4Ma...,NaN,PAID,Europe/London,2024-04-01T10:37:50Z,2024-04-02T21:18:02Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,dY7igwvVpNjrEvmwS42R8ckeV,NaN
3,inv:0-ChCE8YFcEvpKABoi8XJDA3TsEIMP,0,SBBJSGR24YHYB,[{'uid': '8a03de18-0edd-43b4-bbb4-3159f44369ad...,"{'customer_id': 'X3WGSQHHC97PK2KJZ91EBA9HNG', ...",000535,March Training,,https://squareup.com/pay-invoice/inv:0-ChCE8YF...,"{'amount': 2000, 'currency': 'GBP'}",UNPAID,Europe/London,2024-04-01T10:37:14Z,2024-04-01T10:37:15Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN
4,inv:0-ChCXpemgV7lJci13VlhsHtkTEIMP,2,SBBJSGR24YHYB,[{'uid': '2635de4c-476b-4617-ad31-11acb6c0c463...,"{'customer_id': 'DV15NAH2F93BF55AMNSDVHCB20', ...",000534,March Training,,https://squareup.com/pay-invoice/inv:0-ChCXpem...,NaN,PAID,Europe/London,2024-04-01T10:36:01Z,2024-04-01T14:27:43Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,7lCwxkLKlPMsyZXlLjm5zJgeV,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,inv:0-ChDPM4kl4NAiP5pXozC1Y_ESEIMP,1,SBBJSGR24YHYB,[{'uid': '35e32c13-7cdc-449f-b072-478fd634b433...,"{'customer_id': 'GB6X1PRCQZHR7BW76TTDCADACW', ...",000052,,,https://squareup.com/pay-invoice/inv:0-ChDPM4k...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2023-06-29T11:10:01Z,2023-07-03T06:51:24Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,2023-03-30
49,inv:0-ChAUAlcEksn2SoFvemS9qv1AEIMP,0,SBBJSGR24YHYB,[{'uid': '1bfd6dd0-1452-4d9e-905c-556bc79f0ba8...,"{'customer_id': 'DVXQMTVG4MFTHQ765VFCSDCZC8', ...",000053,,,https://squareup.com/pay-invoice/inv:0-ChAUAlc...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2023-06-29T11:44:40Z,2023-07-31T18:13:08Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN
50,inv:0-ChBb4hlJg_JLA3LSygeItKZ3EIMP,1,SBBJSGR24YHYB,[{'uid': '20c3dd06-10de-444b-82e3-0b20104b9048...,"{'customer_id': 'X2KSKJ4GYN0M7G01D6SYKT1BHC', ...",000048,,,https://squareup.com/pay-invoice/inv:0-ChBb4hl...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2023-06-29T10:59:36Z,2023-07-06T12:04:54Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN
51,inv:0-ChBG0g7YrN-4fOCb0OgWpwEXEIMP,0,SBBJSGR24YHYB,[{'uid': 'e20afefe-e2bb-46de-8567-3347f298368c...,"{'customer_id': 'M2WN0A8Z056ECKKPCZ3M1GY88R', ...",000042,,,https://squareup.com/pay-invoice/inv:0-ChBG0g7...,"{'amount': 500, 'currency': 'GBP'}",UNPAID,Europe/London,2023-06-29T10:55:55Z,2023-07-31T18:13:34Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN


In [109]:
body = {
    'query': {
        'filter': {
            'location_ids': [
                'SBBJSGR24YHYB'
            ],
            'customer_ids': [
                'RX8HM0622ZC9YFTRCNW4Y5PPNG'
            ]
        },
        'sort': {
            'field': 'INVOICE_SORT_DATE',
            'order': 'DESC'
        }
    }
}

In [110]:
result = invoices_api.search_invoices(body)
for inv in result.body['invoices']:
    print(inv)

{'id': 'inv:0-ChBTkUYRxJir_jACj0Cys3wiEIMP', 'version': 0, 'location_id': 'SBBJSGR24YHYB', 'payment_requests': [{'uid': '6f28a405-abdc-4652-be6b-a4168118aad1', 'request_type': 'BALANCE', 'due_date': '2024-04-01', 'tipping_enabled': False, 'computed_amount_money': {'amount': 1000, 'currency': 'GBP'}, 'total_completed_amount_money': {'amount': 0, 'currency': 'GBP'}, 'automatic_payment_source': 'NONE'}], 'primary_recipient': {'customer_id': 'RX8HM0622ZC9YFTRCNW4Y5PPNG', 'given_name': 'Ben', 'family_name': 'Dockerill', 'email_address': 'benwd@live.fr', 'phone_number': '+447592518802', 'address': {'address_line_1': '82B Hanley Road', 'address_line_2': '82', 'locality': 'LONDON', 'administrative_district_level_1': 'ENG', 'postal_code': 'N4 3DW', 'country': 'GB'}}, 'invoice_number': '000483', 'title': 'March Training', 'description': '', 'public_url': 'https://squareup.com/pay-invoice/inv:0-ChBTkUYRxJir_jACj0Cys3wiEIMP', 'next_payment_amount_money': {'amount': 1000, 'currency': 'GBP'}, 'statu

# Format new subs and get customer names

In [111]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'updated_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    new_order['tenders']=get_value(order,'tenders')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        pass
        invoice = invoices[invoices['order_id']==new_order['id']]
        
        
        if len(invoice)>0:
            cust_id = list(invoice['primary_recipient'])[0]['customer_id']
            customer = customers_api.retrieve_customer(cust_id)
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)

    if 'line_items' in order: 
        for item in order['line_items']:
            line_dict = new_order.copy()
            
            line_dict['variation_name']=get_value(item,'variation_name')
            line_dict['item_name']=get_value(item,'name')

            # if 'raining' in line_dict['item_name']:
            #     print(item)
            line_dict['note']=get_value(item,'note')
            line_dict['total_money']=item['total_money']['amount']/100
            orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 500
1 / 500
2 / 500
3 / 500
4 / 500
5 / 500
6 / 500
7 / 500
8 / 500
9 / 500
10 / 500
11 / 500
12 / 500
13 / 500
14 / 500
15 / 500
16 / 500
17 / 500
18 / 500
19 / 500
20 / 500
21 / 500
22 / 500
23 / 500
24 / 500
25 / 500
26 / 500
27 / 500
28 / 500
29 / 500
30 / 500
31 / 500
32 / 500
33 / 500
34 / 500
35 / 500
36 / 500
37 / 500
38 / 500
39 / 500
40 / 500
41 / 500
42 / 500
43 / 500
44 / 500
45 / 500
46 / 500
47 / 500
48 / 500
49 / 500
50 / 500
51 / 500
52 / 500
53 / 500
54 / 500
55 / 500
56 / 500
57 / 500
58 / 500
59 / 500
60 / 500
61 / 500
62 / 500
63 / 500
64 / 500
65 / 500
66 / 500
67 / 500
68 / 500
69 / 500
70 / 500
71 / 500
72 / 500
73 / 500
74 / 500
75 / 500
76 / 500
77 / 500
78 / 500
79 / 500
80 / 500
81 / 500
82 / 500
83 / 500
84 / 500
85 / 500
86 / 500
87 / 500
88 / 500
89 / 500
90 / 500
91 / 500
92 / 500
93 / 500
94 / 500
95 / 500
96 / 500
97 / 500
98 / 500
99 / 500
100 / 500
101 / 500
102 / 500
103 / 500
104 / 500
105 / 500
106 / 500
107 / 500
108 / 500
109 / 500
110 / 500


In [112]:
orders_df

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money
0,r43F70CmXvyTt9lNhlyidFdmaTKZY,2024-03-26T12:01:09.000Z,None,COMPLETED,"[{'id': 'hI4EbezPwRGrru8B3bZVmuTPzMNZY', 'loca...",Joel,Wright,Full Game,Match Sub,None,10.0
1,HmvXJU1X9HzqTUaEeA2pLbiE8EYZY,2024-03-22T12:02:17.000Z,None,COMPLETED,"[{'id': 'rhoDbRhjmr7MMhLpWz46yZMyehKZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
2,tTq8I2j0AsbKDGCNCsijyf6eQVbZY,2024-03-22T11:56:19.000Z,None,COMPLETED,"[{'id': 'lmmUpTmGE5X8XFuLb9EWOCuKU0DZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
3,HYQ3I0dlCcWhKWoDNUJ6nnj4xN6YY,2024-03-21T21:44:56.000Z,None,COMPLETED,"[{'id': 'z3hRSZG2KbVBUNJGXcuksyHJ1ZHZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
4,zQtto0boGmLk1rHVKI0vTEGIyKPZY,2024-03-21T15:48:07.000Z,None,COMPLETED,"[{'id': 'l83hSX9ESK0R9zFy4PjYUinl26GZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
...,...,...,...,...,...,...,...,...,...,...,...
828,TgXaW9aZalv0y7UsRxNhd0dEWeYZY,2024-03-18T21:28:34.264Z,None,OPEN,None,NaN,NaN,Full Game,Match Sub,None,10.0
829,pFWx3cuPBQesLBKxegQfGgzjA38YY,2024-03-18T21:28:32.282Z,None,OPEN,None,Corey,Cleave,Full Game,Match Sub,None,10.0
830,zyWVUROlxWtKKUWT8UMgY3Dd6zTZY,2024-03-18T21:28:30.988Z,None,OPEN,None,Dan,Russell,Full Game,Match Sub,None,5.0
831,348LaZjNIFVxH3aerm8frXpYFPaZY,2024-03-18T21:28:29.068Z,None,OPEN,None,NaN,NaN,Full Game,Match Sub,None,10.0


In [113]:
list(orders_df[orders_df['item_name']=='Training']['tenders'])[0]

[{'id': 'vH4kyMB9bWMUymMS6Z8dFACGCuDZY',
  'location_id': 'SBBJSGR24YHYB',
  'transaction_id': 'JjxHB44KvMJqr4FMAb3XQWpqnk9YY',
  'created_at': '2024-03-19T08:51:36Z',
  'amount_money': {'amount': 500, 'currency': 'GBP'},
  'type': 'CARD',
  'card_details': {'status': 'CAPTURED',
   'card': {'card_brand': 'AMERICAN_EXPRESS',
    'last_4': '2008',
    'fingerprint': 'sq-1-BvjBQyTTSskS7KWPSrTSCb5aMaC82VAqpixqoTpM2XeEr0_OWMKSedkHt7DUt_3S0Q'},
   'entry_method': 'KEYED'},
  'tip_money': {'amount': 0, 'currency': 'GBP'},
  'payment_id': 'vH4kyMB9bWMUymMS6Z8dFACGCuDZY'}]

In [114]:
orders_df

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money
0,r43F70CmXvyTt9lNhlyidFdmaTKZY,2024-03-26T12:01:09.000Z,None,COMPLETED,"[{'id': 'hI4EbezPwRGrru8B3bZVmuTPzMNZY', 'loca...",Joel,Wright,Full Game,Match Sub,None,10.0
1,HmvXJU1X9HzqTUaEeA2pLbiE8EYZY,2024-03-22T12:02:17.000Z,None,COMPLETED,"[{'id': 'rhoDbRhjmr7MMhLpWz46yZMyehKZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
2,tTq8I2j0AsbKDGCNCsijyf6eQVbZY,2024-03-22T11:56:19.000Z,None,COMPLETED,"[{'id': 'lmmUpTmGE5X8XFuLb9EWOCuKU0DZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
3,HYQ3I0dlCcWhKWoDNUJ6nnj4xN6YY,2024-03-21T21:44:56.000Z,None,COMPLETED,"[{'id': 'z3hRSZG2KbVBUNJGXcuksyHJ1ZHZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
4,zQtto0boGmLk1rHVKI0vTEGIyKPZY,2024-03-21T15:48:07.000Z,None,COMPLETED,"[{'id': 'l83hSX9ESK0R9zFy4PjYUinl26GZY', 'loca...",NaN,NaN,Full Game,Match Sub,None,10.0
...,...,...,...,...,...,...,...,...,...,...,...
828,TgXaW9aZalv0y7UsRxNhd0dEWeYZY,2024-03-18T21:28:34.264Z,None,OPEN,None,NaN,NaN,Full Game,Match Sub,None,10.0
829,pFWx3cuPBQesLBKxegQfGgzjA38YY,2024-03-18T21:28:32.282Z,None,OPEN,None,Corey,Cleave,Full Game,Match Sub,None,10.0
830,zyWVUROlxWtKKUWT8UMgY3Dd6zTZY,2024-03-18T21:28:30.988Z,None,OPEN,None,Dan,Russell,Full Game,Match Sub,None,5.0
831,348LaZjNIFVxH3aerm8frXpYFPaZY,2024-03-18T21:28:29.068Z,None,OPEN,None,NaN,NaN,Full Game,Match Sub,None,10.0


# Get Correct columns and filter out unwanted orders

In [115]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[(orders_df['state'].isin(['COMPLETED','OPEN'])) & (orders_df['tenders'].notnull())][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]
# match_fees_export = match_fees_export[match_fees_export['tenders'].notnull()]


match_fees_export = match_fees_export[(match_fees_export['customer_name'].notnull()) | (match_fees_export['note'].notnull())]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [116]:
match_fees_export.columns = exist.columns

NameError: name 'exist' is not defined

In [117]:
match_fees_export

,id,created_at,variation_name,item_name,customer_name,total_money,note
0,r43F70CmXvyTt9lNhlyidFdmaTKZY,2024-03-26T12:01:09.000Z,Full Game,Match Sub,Joel Wright,10.0,None
168,FrmlM8KHj4ByiPHdn07pz4NhgwJZY,2024-03-21T10:53:25.000Z,None,Match Subs - Half Game,James Timmins,5.0,None
169,Bjv9AW8H2GodgZCWSpUC2whfYodZY,2024-03-21T10:53:00.000Z,None,Match Subs,James Timmins,10.0,None
562,VFJC5JlkkJNveEWY2P8WTVIvlqFZY,2024-03-19T17:47:46.000Z,Full Game,Match Sub,Harry Fripp,10.0,None
774,tbQ3nTFnqQldlh7pDEWQhnOhn55YY,2024-03-19T12:18:35.000Z,Full Game,Match Sub,Matt Powell,10.0,None
809,v8AfTp5UboiJzGyWpohf1gPe2PcZY,2024-03-19T10:47:48.000Z,Full Game,Match Sub,Emi Muhadri,10.0,None
816,JjxHB44KvMJqr4FMAb3XQWpqnk9YY,2024-03-19T08:51:37.000Z,None,Training,Pierce Bennett,5.0,None
817,jyOnm2ZUIiL8usJ8wyjDLkfStgbZY,2024-03-19T08:50:51.000Z,None,Training,Pierce Bennett,5.0,None


# Only upload new transactions

In [85]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]
len(not_in)

NameError: name 'exist' is not defined

In [86]:
not_in

NameError: name 'not_in' is not defined

In [87]:
not_in.fillna('NA', inplace=True)

NameError: name 'not_in' is not defined

In [88]:
not_in.to_csv('manual.csv', index=False)

NameError: name 'not_in' is not defined

# Add to Google sheet

In [89]:
append_to_sheet('Square Subs', SAMPLE_SPREADSHEET_ID_input, not_in)

NameError: name 'not_in' is not defined